<a href="https://colab.research.google.com/github/Ekansh-Sethia/vehicle_detection_CV_YOLO/blob/main/Vehicle_Detection_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics -q

import cv2
import requests
import numpy as np
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# 1. Load the newest YOLO model (Nano version for extreme speed)
model = YOLO('yolo11n.pt')

def run_detection_on_image_url(image_url):
    # Fetch image
    try:
        resp = requests.get(image_url, stream=True, timeout=10).raw
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        frame = cv2.imdecode(image, cv2.IMREAD_COLOR)
        if frame is None: return
    except Exception as e:
        print(f"Error: {e}")
        return

    # 3. Optimized Inference
    # vehicle_classes: 2 (car), 3 (motorcycle), 5 (bus), 7 (truck)
    results = model.predict(
        source=frame,
        classes=[2, 3, 5, 7],
        conf=0.25,      # Adjusted threshold for better recall
        save=False,
        verbose=False
    )

    # 4. Result Processing
    # Use the built-in .plot() method which is written in C++ (much faster than Python loops)
    annotated_frame = results[0].plot(line_width=1, font_size=0.5)

    # Add a clean counter
    count = len(results[0].boxes)
    cv2.putText(annotated_frame, f'Vehicles: {count}', (20, 40),
                cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 0, 255), 1)

    # 5. Display
    cv2_imshow(annotated_frame)


if __name__ == "__main__":

    example_image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTK0j4R95WiLCP5HonXA2ttC-Ocybay2exX7w&s"
    run_detection_on_image_url(example_image_url)



